In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# 计算高程

In [ ]:
# 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
# roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [ ]:
# 计算区域高程
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation').clip(roi)
minmax = elevation.reduceRegion(**{
    'reducer': ee.Reducer.minMax(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(minmax.getInfo())

# 计算降雨

In [3]:
# 大庆市
# roi = ee.Geometry.Rectangle([123.9355, 45.8362,125.3055, 46.8762])
# 天津市
# roi = ee.Geometry.Rectangle([115.9244, 38.6272,117.4944,39.6372])
# 武汉市
# roi = ee.Geometry.Rectangle([113.7393, 29.8642,115.0993, 30.9242])
# 珠三角
# roi = ee.Geometry.Rectangle([112.7614, 22.2347,114.0514, 23.2547])
# 青藏高原
roi = ee.Geometry.Rectangle([89.8826, 34.6579,91.3626, 35.8279])
Map.addLayer(roi, {}, "roi")
Map.centerObject(roi,7)

In [4]:
import numpy as np
total_precip = []
for year in range(1999,2020,1):
    startDate = str(year) + '-01-01'
    endDate = str(year) + '-12-31'
    dataset = ee.ImageCollection("ECMWF/ERA5/MONTHLY").select('total_precipitation')
    precip = dataset.filterDate(startDate, endDate).filterBounds(roi).sum()
    mean = precip.reduceRegion(**{
        'reducer': ee.Reducer.mean(),
        'geometry': roi,
        'scale': 30,
        'maxPixels': 1e14,
    })
    # print('{} year: {}'.format(year,mean.get('total_precipitation').getInfo()))
    total_precip.append(mean.get('total_precipitation').getInfo())
print(total_precip)
a = np.mean(total_precip)
print(a * 1000)

[0.5549607160452371, 0.5264014813344923, 0.5416603381219236, 0.579788105057839, 0.5136030718201744, 0.5163553066014391, 0.5656145445217168, 0.5571801750139909, 0.49250266632042256, 0.5239975167971547, 0.6001935464041819, 0.588218146880009, 0.5340043401782483, 0.6040264793323109, 0.501744651064556, 0.5567782024821686, 0.43643440644438086, 0.5908823809989177, 0.6077103785696186, 0.6500532677273545, 0.5442086784963965]
551.7294476291684


In [ ]:
## 数据地址
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_MONTHLY#description

In [ ]:
# 大庆市
468.64445781233803
# 天津市
569.5559105292251
# 武汉市
1382.479354558
# 珠三角
1880.7383048600047
# 青藏高原
551.7294476291684

In [ ]:
dataset = ee.ImageCollection('NASA/GPM_L3/IMERG_MONTHLY_V06').select('precipitation').filterDate('2019-01-01', '2019-12-31').filterBounds(roi)
precip = dataset.map(lambda i : i.multiply(0.72))
mean = precip.sum().reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': roi,
    'scale': 30,
    'maxPixels': 1e14,
})
print(mean.getInfo())